In [ ]:
!pip install protobuf==3.20.* --force-reinstall

In [ ]:
pip install -U transformers datasets accelerate sentencepiece evaluate

In [ ]:
import torch
torch.cuda.get_device_name(0)

In [ ]:
import pandas as pd
from datasets import Dataset

df = pd.read_csv('/kaggle/input/llm-judge/train 2.csv')
df.head()

In [ ]:
def convert_label(row):
    if row["winner_model_a"] == 1:
        return 0
    elif row["winner_model_b"] == 1:
        return 1
    else:
        return 2   

def build_input_text(prompt, a, b):
    return (
        "<|prompt|>\n" + str(prompt) + "\n\n"
        "<|response_a|>\n" + str(a) + "\n\n"
        "<|response_b|>\n" + str(b)
    )

rows = []
for i, row in df.iterrows():
    label = convert_label(row)

    rows.append({
        "id": row["id"],
        "input_text": build_input_text(row["prompt"], row["response_a"], row["response_b"]),
        "label": label
    })

# Save prepared dataset
pd.DataFrame(rows).to_csv("train_prepared.csv", index=False)

print("Saved train_prepared.csv")

In [ ]:
df1 = pd.read_csv('/kaggle/working/train_prepared.csv') 
dataset = Dataset.from_pandas(df1)
dataset = dataset.class_encode_column("label")

In [ ]:
train_test = dataset.train_test_split(test_size=0.05, seed=42)
train_dataset = train_test["train"]
eval_dataset = train_test["test"]

In [ ]:
from transformers import AutoTokenizer

MODEL_NAME = "microsoft/deberta-v3-base"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def tokenize(batch):
    return tokenizer(
        batch["input_text"],
        padding=False,
        truncation=True,
        max_length=512
    )

train_dataset = train_dataset.map(tokenize, batched=True)
eval_dataset  = eval_dataset.map(tokenize, batched=True)

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=3
)

In [ ]:
from transformers import TrainingArguments, IntervalStrategy

training_args = TrainingArguments(
    output_dir="deberta-pref",
    eval_strategy=IntervalStrategy.EPOCH,
    save_strategy=IntervalStrategy.EPOCH,
    logging_strategy="steps",
    logging_steps=100,

    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=16,

    num_train_epochs=1,
    learning_rate=2e-5,
    warmup_ratio=0.05,
    weight_decay=0.01,

    fp16=True,
    bf16=False,
    optim="adamw_torch",
    report_to="none",
)

In [ ]:
from transformers import Trainer, DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

In [ ]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()


trainer.train()

In [ ]:
trainer.save_model("deberta-pref-final")
tokenizer.save_pretrained("deberta-pref-final")

In [ ]:
!zip -r deberta_model.zip /kaggle/working/deberta-pref-final